In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

In [3]:
pdf_folder_path = r"D:\Internship\EnterpriseKnowledgeBased\data\pdfs"

documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        # loader = UnstructuredPDFLoader(pdf_path)
        documents.extend(loader.load())

#spilt the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)





d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [4]:
documents

[Document(metadata={'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Basic-Required-Documents-for-Auto-Loan.pdf', 'page': 0}, page_content='Auto Loan  \nCommon D ocuments for all S egments  \n\uf0b7 4 copies of lab print Passport size photograph of Loan applicant  \n\uf0b7 Photocopy of NID /Smart card  of Loan applicant  \n\uf0b7 Visiting C ard/ Business Card/ Office ID copy of loan applicant  \n\uf0b7 Valid Car quotation  \n\uf0b7 Latest E-TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Letter of Introduction (LOI) /Salary Certificate/Pay Slip  \nDocuments for Self -Employed/Profession als like Doctors, Engineers, Accountants, \nArchitects etc.  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Self-income declaration on professional Pad, Letter head  \n\uf0b7 Latest Professional Qualif ication/ Membership Ce

In [5]:
split_documents

[Document(metadata={'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Basic-Required-Documents-for-Auto-Loan.pdf', 'page': 0}, page_content='Auto Loan  \nCommon D ocuments for all S egments  \n\uf0b7 4 copies of lab print Passport size photograph of Loan applicant  \n\uf0b7 Photocopy of NID /Smart card  of Loan applicant  \n\uf0b7 Visiting C ard/ Business Card/ Office ID copy of loan applicant  \n\uf0b7 Valid Car quotation  \n\uf0b7 Latest E-TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person  \n\uf0b7 Last 6 month’s Bank Statement'),
 Document(metadata={'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Basic-Required-Documents-for-Auto-Loan.pdf', 'page': 0}, page_content='Documents for Salaried Person  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Letter of Introduction (LOI) /Salary Certificate/Pay Slip  \nDocuments for Self -Employed/Professi

In [6]:

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [6]:
from qdrant_client import QdrantClient,models
url = 'http://localhost:6333'
collection_name  = "pdf_db"

client = QdrantClient(url)
client.create_collection(
    collection_name = collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE,
    ),
)

ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
# #uploding the content in the collection 
# client.upload_points(
#     collection_name="pdf_db",
#     points=[
#         models.PointStruct(
#             id=idx, 
#             vector=embeddings.embed_query(split_documents.page_content),  # Extract the text content
#             payload={"text": split_documents.page_content, "metadata": split_documents.metadata}  # Create a payload dictionary
#         )
#         for idx, doc in enumerate(split_documents)  # Use split_documents here
#     ],
# )

# Filter out documents with empty or None content
filtered_documents = [doc for doc in split_documents if doc.page_content]
print(len(filtered_documents))
# Now use the filtered documents for further processing
points = []
for idx, doc in enumerate(filtered_documents):
    vector = embeddings.embed_query(doc.page_content)  # Convert the document to a vector
    points.append(
        models.PointStruct(
            id=idx,  # Unique ID for the document
            vector=vector,  # The embedding vector
            payload={"text": doc.page_content}  # Store the text content as payload
        )
    )

# Perform the upsert operation
client.upsert(
    collection_name=collection_name,
    points=points
)



21


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
filtered_documents

[Document(metadata={'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Basic-Required-Documents-for-Auto-Loan.pdf', 'page': 0}, page_content='Auto Loan  \nCommon D ocuments for all S egments  \n\uf0b7 4 copies of lab print Passport size photograph of Loan applicant  \n\uf0b7 Photocopy of NID /Smart card  of Loan applicant  \n\uf0b7 Visiting C ard/ Business Card/ Office ID copy of loan applicant  \n\uf0b7 Valid Car quotation  \n\uf0b7 Latest E-TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Letter of Introduction (LOI) /Salary Certificate/Pay Slip  \nDocuments for Self -Employed/Profession als like Doctors, Engineers, Accountants, \nArchitects etc.  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Self-income declaration on professional Pad, Letter head  \n\uf0b7 Latest Professional Qualif ication/ Membership Ce

In [ ]:
#search query 
def search_query(query:str):
    query_vector = embeddings.embed_query(query)
    search_results = client.search(
        collection_name = collection_name,
        query_vector = query_vector,
        query_filter=None,
        limit = 5
    )
    return search_results

In [ ]:
response = search_query("What are the required documents needed for student loan")
response

[ScoredPoint(id=20, version=3, score=0.62012184, payload={'text': 'Student  Loan  \n \nCommon Documents for all Segments  \n\uf0b7 Photocopy of NID/Smart card of Loan applicant and guarantor  \n\uf0b7 3 copies of Passport size lab print photograph of  Loan applicant and 2 copies of guarantor  \n\uf0b7 Visiting Card/Business card/Office ID copy of both loan applicant and guarantor  \n\uf0b7 Latest E -TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person  \n\uf0b7 Letter of Introductio n (LOI)/Salary Certificate/Pay Slip  \n\uf0b7 Last 6 month’s Bank Statement  \nDocuments for Self -Employed/Professionals like Doctors, Engineers, Accountants, Architects etc.  \n\uf0b7 Last 6 month’s Bank Statement  \n\uf0b7 Latest Professional Qualification/ Membership Certificate f or self -professional  \n\uf0b7 Self-income declaration o n professional Pad, Letter head  \nDocuments for Businessma

## MARKDOWN PDF READER

## CHROMA DB


In [7]:
import chromadb
from langchain.vectorstores import Chroma
import json

def create_chroma_db(split_documents, encoder):
    # Initialize ChromaDB client
    # chroma_client = chromadb.Client()
    
    # # Configuration for vector storage (adjust according to your needs)
    # vectors_config = {
    #     'dimensions': encoder.get_sentence_embedding_dimension(),  # Ensure this method correctly returns the dimension
    #     'distance': 'cosine',  # or 'euclidean'
    #     'index_type': 'hnsw'  # Ensure this index type is supported by ChromaDB
    # }
    # json_vectors_config = json.dumps(vectors_config)
    
    # Attempt to retrieve the specific collection
    # try:
    #     pdf_collection = chroma_client.get_collection("pdf_db")
    #     print("Collection already exists.")
    # except Exception as e:
    #     # If the collection does not exist, create it
    #     print("Collection does not exist. Creating new collection...")
    #     pdf_collection = chroma_client.create_collection("pdf_db", json_vectors_config)
    
    # Create a Chroma vector store from documents
    vectordb = Chroma.from_documents(
        documents=split_documents,
        embedding=embeddings,
        persist_directory=r"D:\Internship\EnterpriseKnowledgeBased\chroma_store"
    )
    vectordb.persist()
    
    return vectordb

# Example usage (ensure split_documents and encoder are properly defined)
# create_chroma_db(split_documents, encoder)


In [8]:
chroma_db = create_chroma_db(split_documents,embeddings)

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores.qdrant import Qdrant
from langchain_community.llms import CTransformers
from langchain_core.prompts import PromptTemplate


llm = CTransformers(
    model='D:/Internship/EnterpriseKnowledgeBased/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {input}

Only return the helpful answer. Answer must be detailed and well explained.
Helpful answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'input'])

# city_bank_faq_db_retriever = Qdrant(client=client, embeddings=embeddings, collection_name="pdf_db")
# retriever = city_bank_faq_db_retriever.as_retriever()


#chromadb retirever
retriever = chroma_db.as_retriever()


combine_docs_chain = create_stuff_documents_chain(
    llm=llm,
    # Default prompt is loaded from the hub, but we can also modify it
    prompt=prompt,
    # prompt=hub.pull("langchain-ai/retrieval-qa-chat")
)
retrieval_qa_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain,
)
response = retrieval_qa_chain.invoke({"input":"What are the required documents needed for student loan"})
print(response)

{'input': 'What are the required documents needed for student loan', 'context': [Document(metadata={'page': 0, 'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Student-Loan-Required-Documents.pdf'}, page_content='Student  Loan  \n \nCommon Documents for all Segments  \n\uf0b7 Photocopy of NID/Smart card of Loan applicant and guarantor  \n\uf0b7 3 copies of Passport size lab print photograph of  Loan applicant and 2 copies of guarantor  \n\uf0b7 Visiting Card/Business card/Office ID copy of both loan applicant and guarantor  \n\uf0b7 Latest E -TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person'), Document(metadata={'page': 0, 'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Student-Loan-Required-Documents.pdf'}, page_content='Student  Loan  \n \nCommon Documents for all Segments  \n\uf0b7 Photocopy of NID/Smart card of Loan applicant and guaranto

In [12]:
response

{'input': 'What are the required documents needed for student loan',
 'context': [Document(metadata={'page': 0, 'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Student-Loan-Required-Documents.pdf'}, page_content='Student  Loan  \n \nCommon Documents for all Segments  \n\uf0b7 Photocopy of NID/Smart card of Loan applicant and guarantor  \n\uf0b7 3 copies of Passport size lab print photograph of  Loan applicant and 2 copies of guarantor  \n\uf0b7 Visiting Card/Business card/Office ID copy of both loan applicant and guarantor  \n\uf0b7 Latest E -TIN of customer  \n\uf0b7 Latest Utility bill copy  \n\uf0b7 Sanction letter and repayment state ment of existing loans (if any)  \nDocuments for Salaried Person'),
  Document(metadata={'page': 0, 'source': 'D:\\Internship\\EnterpriseKnowledgeBased\\data\\pdfs\\Student-Loan-Required-Documents.pdf'}, page_content='Student  Loan  \n \nCommon Documents for all Segments  \n\uf0b7 Photocopy of NID/Smart card of Loan applicant and guara